Generación de texto con un RNN 
https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


Leer los datos

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


Procesar el texto
Vectorizar el texto
Antes del entrenamiento, debe convertir las cadenas en una representación numérica.

La capa tf.keras.layers.StringLookup puede convertir cada carácter en un ID numérico. Solo necesita que el texto se divida en tokens primero.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Dado que el objetivo de este tutorial es generar texto, también será importante invertir esta representación y recuperar cadenas legibles por humanos a partir de ella. Para esto, puede usar tf.keras.layers.StringLookup(..., invert=True) .

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

La tarea de predicción
Dado un carácter, o una secuencia de caracteres, ¿cuál es el próximo carácter más probable? Esta es la tarea para la que está entrenando al modelo. La entrada al modelo será una secuencia de caracteres, y usted entrena el modelo para predecir la salida: el siguiente carácter en cada paso de tiempo.

Dado que los RNN mantienen un estado interno que depende de los elementos vistos anteriormente, dados todos los caracteres computados hasta este momento, ¿cuál es el siguiente carácter?

Cree ejemplos y objetivos de capacitación
A continuación, divida el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá caracteres seq_length del texto.

Para cada secuencia de entrada, los objetivos correspondientes contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha.

Así que divide el texto en partes de seq_length+1 . Por ejemplo, digamos que seq_length es 4 y nuestro texto es "Hola". La secuencia de entrada sería "Hell" y la secuencia de destino "ello".

Para hacer esto, primero use la función tf.data.Dataset.from_tensor_slices para convertir el vector de texto en una secuencia de índices de caracteres.

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Para el entrenamiento, necesitará un conjunto de datos de (input, label) pares. Donde input y label son secuencias. En cada paso de tiempo, la entrada es el carácter actual y la etiqueta es el siguiente carácter.

Aquí hay una función que toma una secuencia como entrada, la duplica y la cambia para alinear la entrada y la etiqueta para cada paso de tiempo:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Crear lotes de entrenamiento
Usó tf.data para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, debe mezclar los datos y empaquetarlos en lotes.

In [31]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

construir el modelo
Esta sección define el modelo como una subclase keras.Model (para obtener más información, consulte Creación de nuevas capas y modelos a través de subclases ).

Este modelo tiene tres capas:

tf.keras.layers.Embedding : La capa de entrada. Una tabla de búsqueda entrenable que asignará cada ID de carácter a un vector con dimensiones embedding_dim ;
tf.keras.layers.GRU : un tipo de RNN con units=rnn_units (también puede usar una capa LSTM aquí).
tf.keras.layers.Dense : la capa de salida, con salidas vocab_size . Produce un logit por cada carácter del vocabulario. Estos son el log-verosimilitud de cada carácter según el modelo.

In [32]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [33]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [34]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Prueba el modelo
Ahora ejecute el modelo para ver que se comporta como se esperaba.

Primero verifique la forma de la salida:

In [35]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch)
    print(target_example_batch)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[[21 18 32 ... 55 55 40]
 [14 32 33 ...  9  1  1]
 [57  7  2 ... 14 15 31]
 ...
 [41 44  2 ... 48 58  2]
 [48 51 51 ... 47 48 52]
 [16 51 40 ...  2 41 48]], shape=(64, 100), dtype=int64)
tf.Tensor(
[[18 32 32 ... 55 40 57]
 [32 33 22 ...  1  1 26]
 [ 7  2 22 ... 15 31 14]
 ...
 [44  2 45 ... 58  2 46]
 [51 51  2 ... 48 52  2]
 [51 40 60 ... 41 48 59]], shape=(64, 100), dtype=int64)
(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [36]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [38]:
sampled_indices

array([38, 38, 63, 59, 50,  9, 56, 41, 21, 20, 23, 63, 57,  6,  0, 12, 48,
       44, 53, 51, 34, 13, 47, 46, 43, 20, 29, 46, 46, 26, 12, 14, 52, 16,
       57,  5, 16, 51,  5, 50, 37, 23, 25, 50, 60, 53, 60, 41, 39,  0, 50,
       54, 25, 39, 56, 61, 63, 27, 30, 29,  2, 13,  6, 42,  0, 40, 33, 17,
        7, 12,  0, 64, 64, 53, 62, 53, 33, 63, 45,  9, 30, 20, 48, 16, 20,
       30, 49, 65, 65, 18, 52, 47, 41, 12, 15, 21, 32, 42, 38, 56],
      dtype=int64)

In [39]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"HESS OF YORK:\nWhat should you fear?\n'Tis nothing but some bond, that he is enter'd into\nFor gay appa"

Next Char Predictions:
 b"YYxtk.qbHGJxr'[UNK];ienlU?hgdGPggM;AmCr&Cl&kXJLkunubZ[UNK]koLZqvxNQP ?'c[UNK]aTD,;[UNK]yynwnTxf.QGiCGQjzzEmhb;BHScYq"


entrenar al modelo
En este punto, el problema se puede tratar como un problema de clasificación estándar. Dado el estado anterior de RNN y la entrada de este paso de tiempo, prediga la clase del siguiente carácter.

Adjunte un optimizador y una función de pérdida
La función de pérdida estándar tf.keras.losses.sparse_categorical_crossentropy funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que su modelo devuelve logits, debe configurar el indicador from_logits .

In [40]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [41]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1899085, shape=(), dtype=float32)


In [42]:
tf.exp(example_batch_mean_loss).numpy()

66.01675

In [43]:
model.compile(optimizer='adam', loss=loss)

Configurar puntos de control
Use un tf.keras.callbacks.ModelCheckpoint para asegurarse de que los puntos de control se guarden durante el entrenamiento:

In [44]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [45]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 9s 39ms/step - loss: 2.7074
Epoch 2/20
172/172 [==============================] - 7s 41ms/step - loss: 1.9830
Epoch 3/20
172/172 [==============================] - 7s 38ms/step - loss: 1.7067
Epoch 4/20
172/172 [==============================] - 7s 39ms/step - loss: 1.5460
Epoch 5/20
172/172 [==============================] - 7s 39ms/step - loss: 1.4468
Epoch 6/20
172/172 [==============================] - 7s 37ms/step - loss: 1.3791
Epoch 7/20
172/172 [==============================] - 7s 37ms/step - loss: 1.3264
Epoch 8/20
172/172 [==============================] - 7s 37ms/step - loss: 1.2822
Epoch 9/20
172/172 [==============================] - 7s 38ms/step - loss: 1.2410
Epoch 10/20
172/172 [==============================] - 7s 37ms/step - loss: 1.2018
Epoch 11/20
172/172 [==============================] - 7s 38ms/step - loss: 1.1623
Epoch 12/20
172/172 [==============================] - 7s 38ms/step - loss: 1.1211
Epoch 13/20
1

Generar texto
La forma más sencilla de generar texto con este modelo es ejecutarlo en un bucle y realizar un seguimiento del estado interno del modelo a medida que lo ejecuta.
Cada vez que llama al modelo, pasa un texto y un estado interno. El modelo devuelve una predicción para el siguiente carácter y su nuevo estado. Vuelva a pasar la predicción y el estado para continuar generando texto.

Lo siguiente hace una predicción de un solo paso:

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Ejecútelo en un bucle para generar algo de texto. Mirando el texto generado, verá que el modelo sabe cuándo usar mayúsculas, hacer párrafos e imita un vocabulario de escritura similar al de Shakespeare. Con el pequeño número de épocas de entrenamiento, aún no ha aprendido a formar oraciones coherentes.

In [48]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
In'tio, peace:
I'll ha' my meet; and, by my tale, I will
not turned and set for it.

GLOUCESTER:
I will, my diech-lying.

DUKE VINCENTIO:
Well, thou, or when Tybalt, Let him in ut all?
The mayor towards London they learning in your arms,
But lets your slender wanton puts together
And instruments of the masch. What is thy grining would,
And shame so pardon then of Junies,
Are done, but long to hear a worthy nurse,
How he appears dicarred wnought,
I will make foul grief broke. He mean to see your heart;
Whereto we drink the ear that contradict
Is a house and Birbshal quive or other
The man not with myself with my delivery
As they meen and medely draws, in a commoster
Toward the nobilit in my misery, and
your mother had been blood, our caminis lords.

QUEEN ELIZABETH:
She cross, my lord of Lucio.

LUCIO:
Fear esemes we stoop.

MENENIUS:
Why then no more?
She shall not brought the power of unchrobed;
Though his victorious regar to their princes,
now power: the magam warriors, steel 

Lo más fácil que puede hacer para mejorar los resultados es entrenarlo durante más tiempo (pruebe EPOCHS = 30 ).

También puede experimentar con una cadena de inicio diferente, intente agregar otra capa RNN para mejorar la precisión del modelo o ajustar el parámetro de temperatura para generar predicciones más o menos aleatorias.

Si desea que el modelo genere texto más rápido , lo más fácil que puede hacer es generar el texto por lotes. En el siguiente ejemplo, el modelo genera 5 salidas aproximadamente en el mismo tiempo que se tardó en generar 1 arriba.

In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe mayor in hand, as he will desire.\n\nPETRUCHIO:\nI know not what to say that she is fight\nThat hads more matter to shoce me for't.\n\nPETRUCHIO:\nI'll be a mag--but still with self-wealth for a dear;\nSo doth contrard my true lips,\nWhipping in our countrymen and now in the dispatch:\n'Thus stand but that which shall be consumed with curs,\nStabb'd by our liege, with thy sorrow,\nuncantly and outswands unto her face, and who\nTo stand me: throw the time o' the\nking; what, I propees a bawd, anly night\nHad rather be ourself.\n\nLORD STANLAY:\nIf thou be minex, being good speech; tell me\nThe fault's of honour that my more disigns\nWhose tatte's ease to shruk desolt to discupor.\n\nBUCKINGHAM:\nDo not swear, and fair cousan will the king.\n\nRICHMOND:\nWhy, this we long shall you stay time here,\nAnd that thy polsces by Libut's blood.\n\nGREMIO:\nHarsh he that both prince, and fight-door;\nif thou hast stoop'd in my meed, their trades\nWith duty takes.\nFirst,

Exportar el generador
Este modelo de un solo paso se puede guardar y restaurar fácilmente, lo que le permite usarlo en cualquier lugar donde se acepte un tf.saved_model .

In [50]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [51]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
I do desire to him.
To whom Decoursessing herself twic, thou else,
The precious better death had ga


Avanzado: Entrenamiento Personalizado
El procedimiento de entrenamiento anterior es simple, pero no le da mucho control. Utiliza el maestro forzado que evita que las malas predicciones se retroalimenten al modelo, por lo que el modelo nunca aprende a recuperarse de los errores.

Entonces, ahora que ha visto cómo ejecutar el modelo manualmente, ahora implementará el ciclo de entrenamiento. Esto brinda un punto de partida si, por ejemplo, desea implementar el aprendizaje del plan de estudios para ayudar a estabilizar la salida de bucle abierto del modelo.

La parte más importante de un ciclo de entrenamiento personalizado es la función de paso de entrenamiento.

Use tf.GradientTape para rastrear los degradados. Puede obtener más información sobre este enfoque leyendo la guía de ejecución ansiosa .

El procedimiento básico es:

Ejecute el modelo y calcule la pérdida bajo un tf.GradientTape .
Calcule las actualizaciones y aplíquelas al modelo utilizando el optimizador.

In [52]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

La implementación anterior del método train_step sigue las convenciones train_step de Keras . Esto es opcional, pero le permite cambiar el comportamiento del paso de tren y seguir usando los métodos Model.compile y Model.fit de keras.

In [53]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))


In [54]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 8s 35ms/step - loss: 2.7003


O si necesita más control, puede escribir su propio ciclo de entrenamiento personalizado completo:

In [55]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1523
Epoch 1 Batch 50 Loss 2.0583
Epoch 1 Batch 100 Loss 1.9722
Epoch 1 Batch 150 Loss 1.8533

Epoch 1 Loss: 1.9768
Time taken for 1 epoch 6.86 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7950
Epoch 2 Batch 50 Loss 1.7174
Epoch 2 Batch 100 Loss 1.6798
Epoch 2 Batch 150 Loss 1.6261

Epoch 2 Loss: 1.6955
Time taken for 1 epoch 6.49 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5748
Epoch 3 Batch 50 Loss 1.5589
Epoch 3 Batch 100 Loss 1.5058
Epoch 3 Batch 150 Loss 1.4958

Epoch 3 Loss: 1.5367
Time taken for 1 epoch 6.72 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4303
Epoch 4 Batch 50 Loss 1.4210
Epoch 4 Batch 100 Loss 1.4102
Epoch 4 Batch 150 Loss 1.3767

Epoch 4 Loss: 1.4399
Time taken for 1 epoch 6.41 sec
_________________________________________________________________________